In [1]:
import sys
sys.path.append("/home/hugo/projetos-doutorado/mimo_emb_fts/src/")
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from embfts.models.MimoNonStationaryFtsPca import MimoNonStationaryFtsPca
from pyFTS.models.nonstationary import nsfts
from pyFTS.benchmarks import Measures
from pyFTS.benchmarks import Measures
import matplotlib.pyplot as plt
from pyFTS.common import Util
import datetime
import statistics
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score
import math
from embfts.util.DataSetUtil import DataSetUtil


np.seterr(divide='ignore', invalid='ignore')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [2]:
data_set_util = DataSetUtil()

In [3]:
df = pd.read_csv('/home/hugo/projetos-doutorado/mimo_emb_fts/data/ethylene_methane.csv', sep=',')
df = df.loc[0: : 60]
data = df.drop(labels=['Time (seconds)'], axis=1)
data.dropna(inplace=True)
data = data_set_util.clean_dataset(data)
data.head()

,CO conc (ppm),Ethylene conc (ppm),TGS2602,TGS2602.1,TGS2600,TGS2600.1,TGS2610,TGS2610.1,TGS2620,TGS2620.1,TGS2602.2,TGS2602.3,TGS2600.2,TGS2600.3,TGS2610.2,TGS2610.3,TGS2620.2,TGS2620.3
0,0.0,0.0,-50.85,-1.95,-41.82,1.30,-4.07,-28.73,-13.49,-3.25,55139.95,50669.50,9626.26,9762.62,24544.02,21420.68,7650.61,6928.42
60,0.0,0.0,-52.14,6.68,-27.27,7.17,-6.18,-33.25,-10.73,-0.81,22542.37,21791.37,5706.32,5806.53,11000.82,9895.08,4255.55,3863.13
120,0.0,0.0,-28.73,53.51,-25.00,15.81,1.47,-2.77,-6.99,2.93,7708.09,9138.99,3831.30,3946.52,6235.93,5794.75,2760.12,2515.53
180,0.0,0.0,32.17,186.52,-24.52,17.12,22.35,5.70,-2.12,11.41,3588.41,4628.57,2780.83,2845.23,4037.65,3850.63,1945.41,1753.00
240,0.0,0.0,135.77,415.68,-12.35,30.20,64.38,69.32,15.98,27.09,2167.30,2869.98,2116.91,2171.88,2876.99,2832.08,1466.54,1308.25


In [4]:
data.describe()

,CO conc (ppm),Ethylene conc (ppm),TGS2602,TGS2602.1,TGS2600,TGS2600.1,TGS2610,TGS2610.1,TGS2620,TGS2620.1,TGS2602.2,TGS2602.3,TGS2600.2,TGS2600.3,TGS2610.2,TGS2610.3,TGS2620.2,TGS2620.3
count,70138.000000,70138.000000,70138.000000,70138.000000,70138.000000,70138.000000,70138.000000,70138.000000,70138.000000,70138.000000,70138.000000,70138.000000,70138.000000,70138.000000,70138.000000,70138.000000,70138.000000,70138.000000
mean,127.624819,5.320212,2065.646556,506.223965,4372.540409,4799.282607,1895.013479,2214.631155,5086.604573,5385.520250,1186.659772,1219.603599,4871.117858,3926.286022,928.404101,1035.967468,5289.420927,4309.666532
std,160.786353,5.624039,790.244729,1530.159797,1608.070170,1733.094042,439.551905,435.424275,1966.472022,2072.338994,479.897621,454.744514,1707.686096,1364.626534,182.611871,184.928554,1984.146721,1584.056625
min,0.000000,0.000000,-52.140000,-31.150000,-41.820000,1.300000,-6.180000,-33.250000,-13.490000,-3.250000,168.650000,226.190000,876.840000,884.160000,626.450000,688.190000,555.790000,501.310000
25%,0.000000,0.000000,1324.300000,9.120000,2986.547500,3282.527500,1504.165000,1856.650000,3242.880000,3448.320000,934.330000,941.920000,3435.090000,2729.257500,808.150000,910.250000,3642.420000,2806.400000
50%,0.000000,6.670000,1789.310000,38.720000,4493.640000,4947.450000,1983.850000,2272.660000,5225.630000,5538.690000,1139.520000,1175.890000,5017.230000,4119.755000,929.860000,1060.900000,5320.550000,4543.040000
75%,266.670000,10.000000,2868.900000,158.900000,5658.690000,6178.210000,2262.250000,2566.220000,6719.280000,7103.720000,1543.660000,1558.000000,6126.830000,4967.020000,1033.440000,1161.570000,6739.320000,5565.070000
max,533.330000,20.000000,3541.990000,28064.560000,8597.890000,9196.400000,2888.880000,3476.050000,9861.000000,10484.780000,55139.950000,50669.500000,9626.260000,9762.620000,24544.020000,21420.680000,10634.760000,7847.490000


In [5]:
# frame as supervised learning
reframed = data_set_util.series_to_supervised_mimo(data, 1, 1)
reframed.head()

,CO conc (ppm)(t-1),Ethylene conc (ppm)(t-1),TGS2602(t-1),TGS2602.1(t-1),TGS2600(t-1),TGS2600.1(t-1),TGS2610(t-1),TGS2610.1(t-1),TGS2620(t-1),TGS2620.1(t-1),...,TGS2620(t),TGS2620.1(t),TGS2602.2(t),TGS2602.3(t),TGS2600.2(t),TGS2600.3(t),TGS2610.2(t),TGS2610.3(t),TGS2620.2(t),TGS2620.3(t)
60,0.0,0.0,-50.85,-1.95,-41.82,1.30,-4.07,-28.73,-13.49,-3.25,...,-10.73,-0.81,22542.37,21791.37,5706.32,5806.53,11000.82,9895.08,4255.55,3863.13
120,0.0,0.0,-52.14,6.68,-27.27,7.17,-6.18,-33.25,-10.73,-0.81,...,-6.99,2.93,7708.09,9138.99,3831.30,3946.52,6235.93,5794.75,2760.12,2515.53
180,0.0,0.0,-28.73,53.51,-25.00,15.81,1.47,-2.77,-6.99,2.93,...,-2.12,11.41,3588.41,4628.57,2780.83,2845.23,4037.65,3850.63,1945.41,1753.00
240,0.0,0.0,32.17,186.52,-24.52,17.12,22.35,5.70,-2.12,11.41,...,15.98,27.09,2167.30,2869.98,2116.91,2171.88,2876.99,2832.08,1466.54,1308.25
300,0.0,0.0,135.77,415.68,-12.35,30.20,64.38,69.32,15.98,27.09,...,43.65,57.46,1531.53,2028.66,1687.94,1733.92,2235.14,2237.82,1143.56,1031.91


In [6]:
reframed.loc[:,' CO conc (ppm)(t-1)']

60         0.0
120        0.0
180        0.0
240        0.0
300        0.0
          ... 
4207980    0.0
4208040    0.0
4208100    0.0
4208160    0.0
4208220    0.0
Name:  CO conc (ppm)(t-1), Length: 70137, dtype: float64

In [7]:
cols = reframed.columns
cols

Index([' CO conc (ppm)(t-1)', ' Ethylene conc (ppm)(t-1)', 'TGS2602(t-1)',
       'TGS2602.1(t-1)', 'TGS2600(t-1)', 'TGS2600.1(t-1)', 'TGS2610(t-1)',
       'TGS2610.1(t-1)', 'TGS2620(t-1)', 'TGS2620.1(t-1)', 'TGS2602.2(t-1)',
       'TGS2602.3(t-1)', 'TGS2600.2(t-1)', 'TGS2600.3(t-1)', 'TGS2610.2(t-1)',
       'TGS2610.3(t-1)', 'TGS2620.2(t-1)', 'TGS2620.3(t-1)',
       ' CO conc (ppm)(t)', ' Ethylene conc (ppm)(t)', 'TGS2602(t)',
       'TGS2602.1(t)', 'TGS2600(t)', 'TGS2600.1(t)', 'TGS2610(t)',
       'TGS2610.1(t)', 'TGS2620(t)', 'TGS2620.1(t)', 'TGS2602.2(t)',
       'TGS2602.3(t)', 'TGS2600.2(t)', 'TGS2600.3(t)', 'TGS2610.2(t)',
       'TGS2610.3(t)', 'TGS2620.2(t)', 'TGS2620.3(t)'],
      dtype='object')

In [8]:
df_forecats_columns = [' CO conc (ppm)(t)', ' Ethylene conc (ppm)(t)', 'TGS2602(t)',
       'TGS2602.1(t)', 'TGS2600(t)', 'TGS2600.1(t)', 'TGS2610(t)',
       'TGS2610.1(t)', 'TGS2620(t)', 'TGS2620.1(t)', 'TGS2602.2(t)',
       'TGS2602.3(t)', 'TGS2600.2(t)', 'TGS2600.3(t)', 'TGS2610.2(t)',
       'TGS2610.3(t)', 'TGS2620.2(t)', 'TGS2620.3(t)']

In [9]:
def sliding_window_pca(data,n_windows,train_size,steps_ahead,transformation):

    result = {
         "window": [],
         "rmse": [],
         "mape": [],
         "mae": [],
         "r2": [],
         "variable":[]
    }
    
    final_result = {
         "window": [],
         "rmse": [],
         "mape": [],
         "mae": [],
         "r2": [],
         "variable":[]
    }

    tam = len(data)
    n_windows = 30
    windows_length = math.floor(tam / n_windows)
    for ct, ttrain, ttest in Util.sliding_window(data, windows_length, train_size, inc=1):
        if len(ttest) > 0:
            
            print('-' * 20)
            print(f'training window {(ct)}')
            
            df_train = ttrain.loc[:,' CO conc (ppm)(t-1)':'TGS2620.3(t-1)']
            df_test = ttest.loc[:,' CO conc (ppm)(t-1)':'TGS2620.3(t-1)']
            df_original = ttest.loc[:,' CO conc (ppm)(t)':'TGS2620.3(t)']
            
            models, data_train = mimo_kpca_nsfts.run_train(df_train,transformation)
            forecast, data_test = mimo_kpca_nsfts.run_test(models,df_test,steps_ahead,transformation)
            
            columns = list(df_forecats_columns)
            df_forecast = pd.DataFrame(forecast,columns=columns)
            
            for col in columns:  
                original = df_original[col].values
                forecast = df_forecast[col].values
#                 original = original[:len(original)-1]
#                 forecast = forecast[1:]
                
                #print("[{0: %H:%M:%S}]".format(datetime.datetime.now()) + f" getting statistics for variable: " + col)
                mae = mean_absolute_error(original,forecast)
                r2 = r2_score(original,forecast)
                #rmse = mean_squared_error(original,forecast,squared=False)
                rmse = Measures.rmse(original,forecast)
                mape = Measures.mape(original,forecast)
                
                #nrmse = cal_nrmse(rmse, original)

                result["rmse"].append(rmse)
                #result["nrmse"].append(nrmse)
                result["mape"].append(mape)
                result["mae"].append(mae)
                result["r2"].append(r2)
                result["window"].append(ct)
                result["variable"].append(col)
                
#                 fig, ax = plt.subplots(nrows=1, ncols=1, figsize=[15, 3])
#                 ax.plot(original, label='Original')
#                 ax.plot(forecast, label='Forecast')
#                 handles, labels = ax.get_legend_handles_labels()
#                 lgd = ax.legend(handles, labels, loc=2, bbox_to_anchor=(1, 1))
#                 plt.show()
        
    measures = pd.DataFrame(result)
    return measures

## Sliding Window: MIMO KPCA-NSFTS

In [19]:
num_components_pca = 10
order_fts_model = 1
npart = 50
gamma = 0.01
fts_model = nsfts.NonStationaryFTS
memory_window_error = 5
steps_ahead = 1

mimo_kpca_nsfts = MimoNonStationaryFtsPca(num_components_pca, order_fts_model, npart,
                                            gamma, fts_model, memory_window_error)

In [20]:
kpca_result =  sliding_window_pca(data=reframed,n_windows=30,train_size=0.75,steps_ahead=1,transformation='KPCA')

--------------------
training window 0
--------------------
training window 2337
--------------------
training window 4674
--------------------
training window 7011
--------------------
training window 9348
--------------------
training window 11685
--------------------
training window 14022
--------------------
training window 16359
--------------------
training window 18696
--------------------
training window 21033
--------------------
training window 23370
--------------------
training window 25707
--------------------
training window 28044
--------------------
training window 30381
--------------------
training window 32718
--------------------
training window 35055
--------------------
training window 37392
--------------------
training window 39729
--------------------
training window 42066
--------------------
training window 44403
--------------------
training window 46740
--------------------
training window 49077


/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100


--------------------
training window 51414
--------------------
training window 53751


/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100


--------------------
training window 56088
--------------------
training window 58425
--------------------
training window 60762
--------------------
training window 63099
--------------------
training window 65436
--------------------
training window 67773


In [21]:
columns = list(df_forecats_columns)

final_result = {
    "variable": [],
    "rmse": [],
    "mae": [],
    "mape": [],
    "r2": []
}

measures = kpca_result
var = measures.groupby("variable")

for col in columns:
    
    var_agr = var.get_group(col)
           
    rmse = round(statistics.mean(var_agr.loc[:,'rmse']),3)
    mape = round(statistics.mean(var_agr.loc[:,'mape']),3)
    mae = round(statistics.mean(var_agr.loc[:,'mae']),3)
    r2 = round(statistics.mean(var_agr.loc[:,'r2']),3)

    final_result["variable"].append(col)
    final_result["rmse"].append(rmse)
    final_result["mape"].append(mape)
    final_result["mae"].append(mae)
    final_result["r2"].append(r2)
        
    #print(f'Results: {(col,rmse,mae,r2)}')
        
        
final_measures_kpca = pd.DataFrame(final_result) 



In [22]:
print("Statistics MIMO KPCA-NSFTS(test): ")
round(final_measures_kpca,3)

Statistics MIMO KPCA-NSFTS(test): 


,variable,rmse,mae,mape,r2
0,CO conc (ppm)(t),11.825,2.283,NaN,-3.998506e+20
1,Ethylene conc (ppm)(t),0.393,0.080,NaN,-7.142410e+21
2,TGS2602(t),17.494,13.692,0.735,9.940000e-01
3,TGS2602.1(t),39.287,17.116,inf,-2.630000e-01
4,TGS2600(t),38.817,28.578,0.781,9.950000e-01
5,TGS2600.1(t),41.408,30.445,0.759,9.950000e-01
6,TGS2610(t),16.346,12.245,0.687,9.940000e-01
7,TGS2610.1(t),16.781,12.547,0.590,9.940000e-01
8,TGS2620(t),47.582,34.139,0.824,9.950000e-01
9,TGS2620.1(t),50.756,36.499,0.825,9.940000e-01


## Sliding Window: MIMO PCA-NSFTS

In [23]:
num_components_pca = 10
order_fts_model = 1
npart = 50
gamma = 0.1
fts_model = nsfts.NonStationaryFTS
memory_window_error = 3
steps_ahead = 1

mimo_kpca_nsfts = MimoNonStationaryFtsPca(num_components_pca, order_fts_model, npart,
                                            gamma, fts_model, memory_window_error)

In [24]:
pca_result =  sliding_window_pca(data=reframed,n_windows=30,train_size=0.75,steps_ahead=1,transformation='PCA')

--------------------
training window 0
--------------------
training window 2337
--------------------
training window 4674
--------------------
training window 7011
--------------------
training window 9348
--------------------
training window 11685
--------------------
training window 14022
--------------------
training window 16359


/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100


--------------------
training window 18696
--------------------
training window 21033


/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100


--------------------
training window 23370


...

--------------------
training window 53751


/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100


--------------------
training window 56088
--------------------
training window 58425


/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100


--------------------
training window 60762
--------------------
training window 63099
--------------------
training window 65436
--------------------
training window 67773


In [25]:
columns = list(df_forecats_columns)

final_result = {
    "variable": [],
    "rmse": [],
    "mae": [],
    "mape": [],
    "r2": []
}

measures = pca_result
var = measures.groupby("variable")

for col in columns:
    
    var_agr = var.get_group(col)
           
    rmse = round(statistics.mean(var_agr.loc[:,'rmse']),3)
    mape = round(statistics.mean(var_agr.loc[:,'mape']),3)
    mae = round(statistics.mean(var_agr.loc[:,'mae']),3)
    r2 = round(statistics.mean(var_agr.loc[:,'r2']),3)

    final_result["variable"].append(col)
    final_result["rmse"].append(rmse)
    final_result["mape"].append(mape)
    final_result["mae"].append(mae)
    final_result["r2"].append(r2)
        
    #print(f'Results: {(col,rmse,mae,r2)}')
        
        
final_measures_pca = pd.DataFrame(final_result) 

print("Statistics MIMO PCA-NSFTS(test): ")
final_measures_pca

Statistics MIMO PCA-NSFTS(test): 


,variable,rmse,mae,mape,r2
0,CO conc (ppm)(t),24.127,16.555,NaN,0.949
1,Ethylene conc (ppm)(t),0.889,0.627,inf,0.703
2,TGS2602(t),66.257,48.723,2.516,0.965
3,TGS2602.1(t),113.260,86.909,inf,-0.364
4,TGS2600(t),66.051,46.991,1.340,0.992
5,TGS2600.1(t),71.984,51.477,1.340,0.991
6,TGS2610(t),24.428,17.703,1.000,0.989
7,TGS2610.1(t),28.090,20.581,0.971,0.987
8,TGS2620(t),90.387,65.000,1.647,0.990
9,TGS2620.1(t),95.785,68.919,1.640,0.990
